In [ ]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
#import config
import quickstart

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "2_intermediate")
out= os.join(root, "data", "3_clean")

In [ ]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

In [ ]:
# files in clean data google drive
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1jxSI_nq32-W9kPHKvQkzuNGJ5aIBD2j2' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

In [ ]:
# files in intermediate data
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1kiYbPtSYavHQKzyMWtXaLhP-JM2toXRm' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

# Load Fire Dispatch Data

In [ ]:
fires = drive.CreateFile({'id': '1zYSWlWtgNWsruaEkSSCqyXHIIdq7BEAP'})
fires.GetContentFile('fire_dispatch.csv')  

df_fires= pd.read_csv('fire_dispatch.csv').drop("Unnamed: 0", axis = 1)
df_fires.head()

## Load building level data

In [ ]:
csv = drive.CreateFile({'id': '1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G'})
csv.GetContentFile('bld_predictors.csv')  

df_bld= pd.read_csv('bld_predictors.csv').drop("Unnamed: 0", axis = 1)
df_bld.head()

In [ ]:
# drop missing bbl values
df_bld= df_bld.drop(df_bld[df_bld["bbl"].isna()].index, axis = 0)

In [ ]:
## load borobox/bbl crosswalk
box = drive.CreateFile({'id': '1Yf4xIIOBmKU2WIlImat8qaN3z2L239Nw'})
box.GetContentFile('pluto_box.csv')  

df_box= pd.read_csv('pluto_box.csv').drop("Unnamed: 0", axis = 1)
df_box.head()

In [ ]:
df_bld_box= pd.merge(df_bld, df_box[["bbl", "borobox", "dist"]], on = "bbl", indicator = True, how = "outer")

In [ ]:
df_bld_box["_merge"].value_counts()
df_bld_box= df_bld_box.loc[df_bld_box["_merge"] == "both", :]

assert (df_bld_box["_merge"] == "both").all()
df_bld_box.drop("_merge", axis = 1, inplace = True)

In [ ]:
landusecols = ["landuse_" + str(x) for x in df_bld["landuse"].unique()]
bldgclasscols = ["bldgclass_" + str(x) for x in df_bld["bldgclass"].unique()]

df_bld= pd.merge(df_bld, \
                      pd.get_dummies(df_bld[["bbl", "landuse", "bldgclass"]], columns = ["landuse", "bldgclass"]), on = "bbl")

In [ ]:
# load predictors and predictors for acs
df_bld.columns
predictors = ["lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland",
             "assesstot", "exempttot", "yearbuilt", "holc_AB", "holc_CD", "holc_D",
             "tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio",
             "viol_count_per_unit", "hpd_fire_vio_per_unit", "hpd_b_c_vio_per_unit",
             "AL", "NB", "average_energy_usage", "average_energy_usage_mi"]

predictors_acs= df_bld.columns[df_bld.columns.get_loc("totpop"):df_bld.columns.get_loc("crowdingrenter") + 1].tolist()
predictors_full= predictors + predictors_acs + landusecols + bldgclasscols
predictors_full

df_box_sum= df_bld_box.groupby("borobox", as_index = False)[predictors].apply(lambda x: x.mean(skipna = True))

In [ ]:
## load weighted census data
wgt_census = drive.CreateFile({'id': '1H8yM7UH14YdMqi5ASjHMd_dwA4C7-lKe'})
wgt_census.GetContentFile('wgt_census.csv')  

df_wgt_census= pd.read_csv('wgt_census.csv').drop(["Unnamed: 0", "level_0", "index"], axis = 1)
df_wgt_census.head()

In [ ]:
df_box_acs= pd.merge(df_box_sum, df_wgt_census, on = "borobox", indicator = True)
assert all(df_box_acs["_merge"] == "both")

# drop merge var
df_box_acs= df_box_acs.drop("_merge", axis = 1)

## Merge to outcomes

In [ ]:
df_box_final= pd.merge(df_box_acs, df_fires, on = "borobox", indicator = True, how = "left")
df_box_final.head()

In [ ]:
print(df_box_final.shape)
df_box_final["_merge"].value_counts()

assert (df_box_final["_merge"] == "both").all()
df_box_final.drop("_merge", axis = 1, inplace = True)

In [ ]:
## investigate missing values
df_box_final.loc[df_box_final["medianhhincome"].isna(), :]

In [ ]:
# investigate/fill missing values
print(df_box_final.isna().sum(axis = 0).sort_values(ascending = False))

In [ ]:
df_box_final= df_box_final.fillna(0)

## Export analytic file

In [ ]:
df_box_final.to_csv(os.join(root, "data", "3_clean", "analysis_box.csv"))